In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from dataclasses import dataclass
import aiohttp 
import asyncio

In [2]:
@dataclass(frozen=True)
class Job:
    artist_name: str
    composition_name: str
    url: str
        
    @property
    def full_name(self):
        return f"{self.composition_name} by {self.artist_name}"
    
    @property
    def key(self):
        return self.url

In [3]:
def get_url(path):
    return "https://www.hooktheory.com" + path

async with aiohttp.ClientSession() as session:
    async with session.get(get_url("/theorytab/charts/chart/top")) as resp:
        soup = BeautifulSoup(await resp.text())
        top_songs_a_tags = soup.select("ul.grid2468 > li.grid-item > a")

print(f"{len(top_songs_a_tags)} initial song(s).")

49 initial song(s).


In [4]:
def top_song_a_tag_to_job(a):
    href = a.attrs["href"]
    artist_name = a.select_one("tr span.span-song").text[len("by "):]
    composition_name = a.select_one("tr b").text
    return Job(artist_name, composition_name, href)
    
for a in top_songs_a_tags[:10]:
    print(top_song_a_tag_to_job(a))
job = top_song_a_tag_to_job(top_songs_a_tags[10])

Job(artist_name='Queen', composition_name='Bohemian Rhapsody', url='/theorytab/view/queen/bohemian-rhapsody')
Job(artist_name='Daft Punk', composition_name='Get Lucky', url='/theorytab/view/daft-punk/get-lucky')
Job(artist_name='Knife Party', composition_name='Baghdad', url='/theorytab/view/knife-party/baghdad')
Job(artist_name='Celine Dion', composition_name='My Heart Will Go On', url='/theorytab/view/celine-dion/my-heart-will-go-on')
Job(artist_name='Taylor Swift', composition_name='Shake It Off', url='/theorytab/view/taylor-swift/shake-it-off')
Job(artist_name='Green Day', composition_name='Boulevard of Broken Dreams', url='/theorytab/view/green-day/boulevard-of-broken-dreams')
Job(artist_name='Tristam and Braken', composition_name='Frame of Mind', url='/theorytab/view/tristam-and-braken/frame-of-mind')
Job(artist_name='Nintendo', composition_name='The Legend of Zelda Main Theme', url='/theorytab/view/nintendo/legend-of-zelda-main-theme')
Job(artist_name='Deadmau5', composition_name

In [5]:
# SEARCH_AUTH_KEY = input("Search Auth Key: ").strip()
SEARCH_AUTH_KEY = "YHXUiQCa6024e2a88cb48f226a94d16db0c20d993e0a424cfde7834b697445bdf280ce88"

In [6]:
def name_to_dashed_lowercase(name: str):
    return "-".join(e.lower() for e in name.split())
name_to_dashed_lowercase(job.artist_name)

'bob-dylan'

In [7]:
def song_ytlist_a_tag_to_job(a):
    table = a.find_next_sibling()
    return Job(
        artist_name=table.select_one("p.artist").text[len("by "):],
        composition_name=table.select_one("p.song").text, 
        url=a.attrs["href"],
    )

async def process_job(session, job):
    """ :returns (result, new_jobs) """
    payload = {"q": f"{job.artist_name} {job.composition_name}"}
    async with session.post("https://search.hooktheory.com/indexes/theorytabs/search", json=payload) as resp:
        hits = (await resp.json())["hits"]
    
    if not hits:
        return None, []
    
    result = max(hits, key=lambda e: (
        e["song"].lower() == job.composition_name.lower(), 
        e["artist"].lower() == job.artist_name.lower(), 
        len(e["chordAbs"]) + len(e["chordRel"]),
    ))

    artist_in_url = name_to_dashed_lowercase(result["artist"])
    song_in_url = name_to_dashed_lowercase(result["song"])
    url = get_url(f"/theorytab/view/{artist_in_url}/{song_in_url}")
    async with session.get(url) as resp:
        soup = BeautifulSoup(await resp.text())
    
    new_jobs = [
        song_ytlist_a_tag_to_job(a) for a in soup.select("div.col-md-4 > h3 + ul.yt-list a") 
        if a.attrs.get("href", "").startswith("/theorytab/view/")
    ]
    
    return result, new_jobs
    
async with aiohttp.ClientSession(headers={"Authorization": f"Bearer {SEARCH_AUTH_KEY}"}) as session:
    result, new_jobs = await process_job(session, top_song_a_tag_to_job(top_songs_a_tags[12]))
    print(result)
    print(new_jobs)

{'song': 'Four Chord Songs', 'artist': 'The Axis Of Awesome', 'chordAbs': 'qqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqqEqqBqqC#mqqAqq', 'chordRel': 'qqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqqIqqVqqwjqqIVqq', 'SInD': 'qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq1qq5qq6qq4qq', 'key': 'E major', 'id': 1341, 'ytid': 'oOlDewpCfZQ'}
[Job(artist_name='Tim Minchin', composition_name='15 Minutes', url='/theorytab/view/tim-minchin/15-minutes'), Job(artist_name='Tim and Eric', composition_name='Come Over', url='/theorytab/view/tim-and-eric/come-over'), Job(artist_name='Paul and Storm', composition_name='Here

In [11]:
queue = asyncio.Queue()
for a in top_songs_a_tags:
    queue.put_nowait(top_song_a_tag_to_job(a))
print(f"Initial queue size: {queue.qsize()}")    

results = {}
seen = set()
seen_lock = asyncio.Lock()
queued = set()
queued_lock = asyncio.Lock()

Initial queue size: 49


In [17]:
counter_lock = asyncio.Lock()
counter = 10000
n_workers = 100

async def worker(n):
    global counter
    async with aiohttp.ClientSession(headers={"Authorization": f"Bearer {SEARCH_AUTH_KEY}"}) as session: 
        while counter > n_workers:
            job = await queue.get()
            
            async with seen_lock:
                if job.key in seen:
                    continue
                seen.add(job.key)

            try:
                result, new_jobs = await process_job(session, job)
            except Exception as e:
                print(f"{e.__class__.__name__}: {e} for {job}")
                result, new_jobs = None, []

            added_jobs = 0
            async with queued_lock:
                for new_job in new_jobs:
                    if new_job.key not in queued:
                        added_jobs += 1
                        queued.add(new_job.key)
                        queue.put_nowait(new_job)
            
            result["link"] = job.url
            results[job.key] = result

            async with counter_lock:
                counter -= 1
                print(f"{counter:4}\t+{added_jobs}\t{len(new_jobs) - added_jobs:3} skipped\t"
                      f"{len(results):3} total\t{queue.qsize():3} queued\t{job.artist_name}, {job.composition_name}.")
            
            queue.task_done()

workers = [worker(i) for i in range(n_workers)]

import time
start_time = time.time()

# try: 
await asyncio.gather(*workers)
# finally:
#     duration = time.time() - start_time
#     print(f"Done in {duration:.2f} s.")

9999	+0	 48 skipped	10044 total	343 queued	Haken, Cockroach King.
9998	+0	 40 skipped	10045 total	342 queued	Hem, The Seed.
9997	+1	 43 skipped	10046 total	342 queued	C2C, The Beat.
9996	+1	 51 skipped	10047 total	342 queued	AZEDIA, Something.
9995	+0	 41 skipped	10048 total	341 queued	Stupeflip, Stupeflip Vite.
9994	+0	  0 skipped	10049 total	340 queued	Girl's Day, Don't Forget Me.
9993	+0	 51 skipped	10050 total	339 queued	Jon Bellion, Fashion.
9992	+11	 57 skipped	10051 total	349 queued	Leessang, Leessang Blues.
9991	+0	 41 skipped	10052 total	348 queued	Stupeflip, Crou Anthem.
9990	+0	 67 skipped	10053 total	347 queued	The Cure, Lovesong.
9989	+0	 43 skipped	10054 total	346 queued	ShockOne, Light Cycles.
9988	+0	 67 skipped	10055 total	345 queued	The Cure, The Figurehead.
9987	+0	 67 skipped	10056 total	344 queued	The Cure, Pictures Of You.
9986	+0	 44 skipped	10057 total	343 queued	C2C, Down The Road.
9985	+0	 40 skipped	10058 total	342 queued	The Correspondents, Who Knew.
9984	+0

9885	+0	349 skipped	10158 total	254 queued	Lin-Manuel Miranda, Waiting On a Miracle.
9884	+0	 81 skipped	10159 total	253 queued	Bent Van Looy, Shadow of a Man.
9883	+3	 40 skipped	10160 total	255 queued	Kato, Celebrate Life - Stafford Brothers Remix.
9882	+0	 41 skipped	10161 total	254 queued	Geographer, Kites.
9881	+0	 84 skipped	10162 total	253 queued	Perfume, Polyrhythm.
9880	+10	 42 skipped	10163 total	262 queued	Shakey Graves, Dearly Departed.
9879	+1	 64 skipped	10164 total	262 queued	Walk off the Earth, Lightning Bolt.
9878	+0	 84 skipped	10165 total	261 queued	Perfume, Magic of Love.
9877	+0	 84 skipped	10166 total	260 queued	Perfume, Sweet Refrain.
9876	+0	 84 skipped	10167 total	259 queued	Perfume, Omajinai Perori.
9875	+0	 84 skipped	10168 total	258 queued	Perfume, GLITTER.
9874	+0	 84 skipped	10169 total	257 queued	Perfume, Spice.
9873	+0	349 skipped	10170 total	256 queued	Lin-Manuel Miranda, We Know - Hamilton.
9872	+0	 42 skipped	10171 total	255 queued	K Camp, Cut Her Off

9771	+0	 80 skipped	10272 total	212 queued	Jackson 5, I Wanna Be Where You Are.
9770	+2	 43 skipped	10273 total	213 queued	White Town, Your Woman.
9769	+0	 45 skipped	10274 total	212 queued	White Town, Once I Flew.
9768	+0	 80 skipped	10275 total	211 queued	Jackson 5, Rockin Robin.
9767	+0	 80 skipped	10276 total	210 queued	Jackson 5, I Want You Back.
9766	+0	 43 skipped	10277 total	209 queued	Dave Brubeck, Take Five.
9765	+0	  0 skipped	10278 total	208 queued	Yasunori Nishiki, H'aanit the Hunter - Octopath Traveler.
9764	+0	 61 skipped	10279 total	207 queued	Gnarls Barkley, Crazy.
9763	+1	 54 skipped	10280 total	207 queued	Leprous, From The Flame.
9762	+0	 50 skipped	10281 total	206 queued	Yasunori Nishiki, Beneath The Surface.
9761	+0	  0 skipped	10282 total	205 queued	Nataly Dawn, How I Knew Her.
9760	+0	  0 skipped	10283 total	204 queued	Pomplamoose, Don't Stop Lovin Me.
9759	+0	 55 skipped	10284 total	203 queued	Leprous, Rewind.
9758	+0	 50 skipped	10285 total	202 queued	Yasunori 

9660	+0	 50 skipped	10383 total	146 queued	Mike Krol, Like a Star.
9659	+0	 59 skipped	10384 total	145 queued	Injury Reserve, Superman That.
9658	+0	 46 skipped	10385 total	144 queued	IDLES, Model Village.
9657	+0	 49 skipped	10386 total	143 queued	Yves Tumor, Kerosene.
9656	+0	 49 skipped	10387 total	142 queued	Yves Tumor, Licking an Orchid.
9655	+0	 54 skipped	10388 total	141 queued	Chloe Moriondo, Ghost Adventure Spirit Orb.
9654	+0	 40 skipped	10389 total	140 queued	Jockstrap, Acid.
9653	+0	 40 skipped	10390 total	139 queued	SIAMES, Mr Fear.
9652	+0	 48 skipped	10391 total	138 queued	Thomas Bergersen, Dragonland.
9651	+0	  0 skipped	10392 total	137 queued	Snail's House, Snailchan Adventure.
9650	+0	 44 skipped	10393 total	136 queued	The Wonders, That Thing You Do.
9649	+1	 51 skipped	10394 total	136 queued	Julieta Venegas, A Donde Sea.
9648	+0	 49 skipped	10395 total	135 queued	SWMRS, Palm Trees.
9647	+0	 52 skipped	10396 total	134 queued	Julieta Venegas, Bien o Mal.
9646	+0	  0 sk

9550	+0	 47 skipped	10493 total	 57 queued	Mild High Club, Homage.
9549	+0	 50 skipped	10494 total	 56 queued	Ichiko Aoba, Kokoro No Sekai.
9548	+0	 62 skipped	10495 total	 55 queued	Vulfpeck, 1612.
9547	+1	 45 skipped	10496 total	 55 queued	Christie, Yellow River.
9546	+0	  0 skipped	10497 total	 54 queued	Girls' Generation, Genie.
9545	+0	 62 skipped	10498 total	 53 queued	Vulfpeck, Smile Meditation.
9544	+1	114 skipped	10499 total	 53 queued	The Muppets, Mah Na Mah Na.
9543	+0	  0 skipped	10500 total	 52 queued	PinocchioP, Because You're Here.
9542	+0	 62 skipped	10501 total	 51 queued	Vulfpeck, Darwin Derby.
9541	+0	 43 skipped	10502 total	 50 queued	MORTEN, Look Closer.
9540	+0	115 skipped	10503 total	 49 queued	The Muppets, Rainbow Connection.
9539	+0	 50 skipped	10504 total	 48 queued	Ichiko Aoba, Terifuriame.
9538	+0	 41 skipped	10505 total	 47 queued	HOME, Resonance.
9537	+2	 49 skipped	10506 total	 48 queued	Dan Croll, Work.
9536	+0	 62 skipped	10507 total	 47 queued	Vulfpeck

9435	+0	 47 skipped	10608 total	  0 queued	Lake Street Dive, Stop Your Crying.
9434	+0	 89 skipped	10609 total	  0 queued	Dax Johnson, A Moment In The Life Of Me.
9433	+0	 69 skipped	10610 total	  0 queued	Scott Joplin, Treemonisha - Act 3 no 27 - A Real Slow Drag.
9432	+0	 46 skipped	10611 total	  0 queued	Bonobo, Noctuary.
9431	+0	 43 skipped	10612 total	  0 queued	Ashes Remain, Right Here.
9430	+0	 69 skipped	10613 total	  0 queued	Scott Joplin, Treemonisha - Aunt Dinah Has Blowed The Horn.
9429	+0	 40 skipped	10614 total	  0 queued	Danny Howard, Apex.
9428	+18	 60 skipped	10615 total	 18 queued	Chris Christodoulou, Coalescence.
9427	+0	 46 skipped	10616 total	 17 queued	Bonobo, All In Forms.
9426	+0	  0 skipped	10617 total	 16 queued	2Pac, I Ain't Mad At Cha.
9425	+0	142 skipped	10618 total	 15 queued	Gorillaz, The Apprentice.
9424	+0	 42 skipped	10619 total	 14 queued	Marcus Warner, Liberation.
9423	+0	142 skipped	10620 total	  0 queued	Gorillaz, Empire Ants.
9422	+0	142 skipped	1

CancelledError: 

In [67]:
from operator import itemgetter 

df = pd.DataFrame([e for e in results.values() if e])
df = df.drop(["SInD", "id", "ytid"], axis=1)

def map_chords_rel(chords_str):
    result = [e for e in chords_str.replace("slash", "/").replace("w", "v").replace("j", "i").split("qq") if e]
    result = [e.strip() for e in result]
    return result

def map_chords_abs(chords_str):
    result = [e for e in chords_str.replace("slash", "/").split("qq") if e]
    result = [e.strip() for e in result]
    return result

df.chordAbs = df.chordAbs.apply(map_chords_abs)
df.chordRel = df.chordRel.apply(map_chords_rel)
df.key = df.key.str.split()
df["keyTonic"] = df.key.apply(itemgetter(0))
df["keyScale"] = df.key.apply(itemgetter(1))
df = df.drop("key", axis=1)
df = df.set_index(["artist", "song"]).sort_index()
df = df[~df.index.duplicated(keep="first")]
df = df.reset_index()
df

,artist,song,chordAbs,chordRel,link,keyTonic,keyScale
0,'Til Tuesday,Voices Carry,"[G, G/D, G, G/D, G, G/D, G, Bb, Bb/F, Bb, Bb/F...","[I, I64, I, I64, I, I64, I, IV, IV64, IV, IV64...",/theorytab/view/til-tuesday/voices-carry,G,mixolydian
1,070 Shake,Guilty Conscience,"[G, Em7, Cmaj9, Am11, D]","[I, vi7, IV9, ii11, V]",/theorytab/view/070-shake/guilty-conscience,G,major
2,100 gecs,Hand Crushed By A Mallet,"[C#m, F#m, A, G#m, C#m, F#m, A, G#m]","[i, iv, VI, v, i, iv, VI, v]",/theorytab/view/100-gecs/hand-crushed-by-a-mallet,C#,minor
3,100 gecs,Money Machine,"[D#m, B, F#, A#m, D#m, B, F#, A#m, D#m, B, F#,...","[vi, IV, I, iii, vi, IV, I, iii, vi, IV, I, ii...",/theorytab/view/100-gecs/money-machine,F#,major
4,10cc,I'm Not In Love,"[A, Am, G#m, G#/B#, C#m, C#m/B, A, B, E, A/E, ...","[IV, iv, iii, V6, vi, vi42, IV, V, I, IV64, i7...",/theorytab/view/10cc/im-not-in-love,E,major
...,...,...,...,...,...,...,...
10484,twenty one pilots,My Blood,"[Bbm, Ab, Db, Fm, Eb, Ab, Bbm, Ab, Db, Fm, Eb,...","[iv, III, VI, i, VII, III, iv, III, VI, i, VII...",/theorytab/view/twenty-one-pilots/my-blood,F,minor
10485,twenty one pilots,Nico And The Niners,"[Dm, Dm, Dm, Dm, Am, Am, Am, Am, G, G, F, F, D...","[iv, iv, iv, iv, i, i, i, i, VII, VII, VI, VI,...",/theorytab/view/twenty-one-pilots/nico-and-the...,A,minor
10486,twenty one pilots,Pet Cheetah,"[Eb, Gm, F, Dm, Eb, Gm, F, Dm]","[VI, i, VII, v, VI, i, VII, v]",/theorytab/view/twenty-one-pilots/pet-cheetah,G,minor
10487,vistlip,-OZONE-,"[Dbmaj7, Dbmaj7, Dbmaj7, Eb7/Db, Ab, Eb7/G, Fm...","[IV7, IV7, IV7, V42, I, V65, vi7, Vsus4, Isus4...",/theorytab/view/vistlip/-ozone-,Ab,major


In [68]:
# df.to_csv("test4-songs.csv", index=False)

In [63]:
from shared import *
read_songs("test4-songs.csv")

,artist,song,chordAbs,chordRel,link,keyTonic,keyScale
0,'Til Tuesday,Voices Carry,"[G, G/D, G, G/D, G, G/D, G, Bb, Bb/F, Bb, Bb/F...","[I, I64, I, I64, I, I64, I, IV, IV64, IV, IV64...",/theorytab/view/til-tuesday/voices-carry,G,mixolydian
1,070 Shake,Guilty Conscience,"[G, Em7, Cmai9, Am11, D]","[I, vi7, IV9, ii11, V]",/theorytab/view/070-shake/guilty-conscience,G,major
2,100 gecs,Hand Crushed By A Mallet,"[C#m, F#m, A, G#m, C#m, F#m, A, G#m]","[i, iv, VI, v, i, iv, VI, v]",/theorytab/view/100-gecs/hand-crushed-by-a-mallet,C#,minor
3,100 gecs,Money Machine,"[D#m, B, F#, A#m, D#m, B, F#, A#m, D#m, B, F#,...","[vi, IV, I, iii, vi, IV, I, iii, vi, IV, I, ii...",/theorytab/view/100-gecs/money-machine,F#,major
4,10cc,I'm Not In Love,"[A, Am, G#m, G#/B#, C#m, C#m/B, A, B, E, A/E, ...","[IV, iv, iii, V6, vi, vi42, IV, V, I, IV64, i7...",/theorytab/view/10cc/im-not-in-love,E,major
...,...,...,...,...,...,...,...
10484,twenty one pilots,My Blood,"[Bbm, Ab, Db, Fm, Eb, Ab, Bbm, Ab, Db, Fm, Eb,...","[iv, III, VI, i, VII, III, iv, III, VI, i, VII...",/theorytab/view/twenty-one-pilots/my-blood,F,minor
10485,twenty one pilots,Nico And The Niners,"[Dm, Dm, Dm, Dm, Am, Am, Am, Am, G, G, F, F, D...","[iv, iv, iv, iv, i, i, i, i, VII, VII, VI, VI,...",/theorytab/view/twenty-one-pilots/nico-and-the...,A,minor
10486,twenty one pilots,Pet Cheetah,"[Eb, Gm, F, Dm, Eb, Gm, F, Dm]","[VI, i, VII, v, VI, i, VII, v]",/theorytab/view/twenty-one-pilots/pet-cheetah,G,minor
10487,vistlip,-OZONE-,"[Dbmai7, Dbmai7, Dbmai7, Eb7/Db, Ab, Eb7/G, Fm...","[IV7, IV7, IV7, V42, I, V65, vi7, Vsus4, Isus4...",/theorytab/view/vistlip/-ozone-,Ab,major


In [60]:
df[df.keyScale=="major"].chordRel.explode().value_counts().head(10)

I      27230
V      19163
IV     18939
vi     11219
ii      4727
V7      4156
iii     4001
ii7     2779
IV7     2723
vi7     2614
Name: chordRel, dtype: int64

In [54]:
df[df.chordRel.apply(lambda x: " ".join(x).startswith("i "))]

,artist,song,chordAbs,chordRel,link,keyTonic,keyScale
2,100 gecs,Hand Crushed By A Mallet,"[C#m, F#m, A, G#m, C#m, F#m, A, G#m]","[i, iv, VI, v, i, iv, VI, v]",/theorytab/view/100-gecs/hand-crushed-by-a-mallet,C#,minor
5,10cc,The Wall Street Shuffle,"[Dm, Bb, C, Dm, G, G, G, Dm, Bb, C, Dm, Dm, Bb...","[i, VI, VII, i, IV, IV, IV, i, VI, VII, i, i, ...",/theorytab/view/10cc/the-wall-street-shuffle,D,minor
8,2 Unlimited,No Limit,"[Bbm, Bbm, Bbm, Eb, F, Bbm, Bbm, Bbm, Eb, F]","[i, i, i, IV, V, i, i, i, IV, V]",/theorytab/view/2-unlimited/no-limit,Bb,minor
12,24kGoldn,Valentino,"[Am, Dm, Em, Am]","[i, iv, v, i]",/theorytab/view/24kgoldn/valentino,A,minor
13,2Pac,2 of Amerikaz Most Wanted,"[D#m, D#m, D#m, D#m, B, D#m, D#m, D#m, D#m, B,...","[i, i, i, i, VI, i, i, i, i, VI, i, i, i, i, V...",/theorytab/view/2pac/2-of-amerikaz-most-wanted,D#,minor
...,...,...,...,...,...,...,...
10462,miss A,Only You,"[Dm, Am, Bb, F, F/E, Dm, Am, Bb, C, C#o, Dm, A...","[i, v, VI, III, III42, i, v, VI, VII, viio, i,...",/theorytab/view/miss-a/only-you,D,minor
10473,rammstein,Mein Hertz Brennt,"[Dm, Gm, Bb, C, Dm, Gm, Bb, C, Cm, Cm, Cm, Cm,...","[i, iv, VI, VII, i, iv, VI, VII, i, i, i, i, i...",/theorytab/view/rammstein/mein-hertz-brennt,D,minor
10474,scarlxrd,HEART ATTACK,"[Bm, Bm]","[i, i]",/theorytab/view/scarlxrd/heart-attack,B,minor
10476,sia,The Church Of What's Happening Now,"[Em, A, D, G, Em, A, D, C, Bm]","[i, IV, VII, III, i, IV, VII, bVI, v]",/theorytab/view/sia/the-church-of-whats-happen...,E,dorian
